In [3]:
#libraries
import keras
import tensorflow as tf
#tf.executing_eagerly()
from keras import backend as K
import cv2
import os
import numpy as np
from keras.optimizers import Adam,SGD
#from keras.optimizers import rmsprop
from keras.models import model_from_json, load_model
from keras.layers import Input, Dense
from keras.models import Model,Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Convolution2D #as Conv2D
from keras.layers.convolutional import Deconv2D as Conv2DTranspose
from keras.layers import Lambda, Input, Dense, MaxPooling2D, BatchNormalization,Input
from keras.layers import UpSampling2D, Dropout, Flatten, Reshape, RepeatVector, LeakyReLU,Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
#from keras.losses import mse, binary_crossentropy
keras.callbacks.TerminateOnNaN()
seed = 7
np.random.seed(seed)
from keras.callbacks import CSVLogger
print('finished')

finished


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"#Setting the script to run on GPU:1,2
print('finished')

finished


In [5]:
import os
import sys
import cv2
import csv
import glob
import numpy as np
import time
from sklearn.utils import shuffle

# path to USB
USBPath = "/home/scope/Carla/CARLA_0.9.6/PythonAPI/new/dataset/"
trainingFolders = ["road1_rgb_center","road1_rgb_center1","road1_rgb_left","road1_rgb_left1","road1_rgb_right","road1_rgb_right1"]
outputcsvs = ["road1_rgb_center.csv","road1_rgb_center1.csv","road1_rgb_left.csv","road1_rgb_left1.csv","road1_rgb_right.csv","road1_rgb_right1.csv"]
history = CSVLogger(USBPath + 'model/kerasloss.csv', append=True, separator=';')
path = 'Test1'

#Load complete input images without shuffling
def load_images():
    inputs = []
    numImages = len(glob.glob1(path,'*.jpg'))
    for i in range(0, numImages):
    #for img in glob.glob(path+'*.png'):
        img = cv2.imread(path+"/frame%d.jpg"%i) #path+"/frame%d.png"%i
        #print(img)
        img = cv2.resize(img, (200,66))
        img = img /255.
        inputs.append(img)
    #inpu = shuffle(inputs)
    #print(inputs)
    print("Total number of images:%d" %(numImages))
    return np.array(inputs)

# def createFolderPaths(folders):
#     paths = []
#     for folder in folders:
#         path = USBPath + folder + '/'
#         paths.append(path)
#     return paths

# def load_training_images():
#     paths = createFolderPaths(trainingFolders)
#     return load_images(paths)

def load_steering_value():
    miny= -1
    maxy= 1
    Y=[]
    dataset = path + '/ProcessData.csv'
    with open(dataset, 'rt') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
            output=[]
            x=(float(row[1])-miny)/(maxy-miny)
            output.append(x)
            Y.append(output)
    return Y       

print('finished')

finished


In [6]:
#Loading images from the datasets
image_input = load_images()     #load_training_images()
steer_output = load_steering_value()
print("Total Steering values:%d"%len(steer_output))
#image_input = np.array(image_input)
steer_output = np.array(steer_output)
print(len(image_input))
print(len(steer_output))
print('finished')

Total number of images:3000
Total Steering values:3000
3000
3000
finished


In [7]:
def createModel():
    # NVidia DAVE-2
    dave2 = Sequential()

    # batch regularization to reduce internal covariate shift
    dave2.add(BatchNormalization(input_shape=(66, 200, 3)))

    # 200x66x3 -> 98x31x24, 5x5 convolution
    dave2.add(Convolution2D(24, 5, 5, border_mode="valid", input_shape=(66, 200, 3), dim_ordering="tf"))
    dave2.add(Activation("relu"))
    dave2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode="same"))

    # 98x31x24 -> 47x14x36, 5x5 convolution
    dave2.add(Convolution2D(36, 5, 5, border_mode="valid"))
    dave2.add(Activation("relu"))
    dave2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode="same"))

    # 47x14x36 -> 22x5x48, 5x5 convolution
    dave2.add(Convolution2D(48, 5, 5, border_mode="valid"))
    dave2.add(Activation("relu"))
    dave2.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode="same"))

    # 22x5x48 -> 20x3x64, 3x3 convolution
    dave2.add(Convolution2D(64, 3, 3, border_mode="valid"))
    dave2.add(Activation("relu"))
    dave2.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), border_mode="same"))

    # 20x3x64 -> 18x1x64, 3x3 convolution
    dave2.add(Convolution2D(64, 3, 3, border_mode="valid"))
    dave2.add(Activation("relu"))
    dave2.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), border_mode="same"))

    # flatten
    dave2.add(Flatten())

    # dense 1164
    dave2.add(Dense(1164))
    dave2.add(Activation("relu"))
    dave2.add(Dropout(0.5))

    # dense 100
    dave2.add(Dense(100))
    dave2.add(Activation("relu"))

    # dense 50
    dave2.add(Dense(50))

    # dense 10
    dave2.add(Dense(10))

    # dense 1
    dave2.add(Dense(1))

    #dave2.compile(loss="mse", optimizer=Adam(LEARNING_RATE))

    return dave2

def trainModel(model, X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15,random_state=42)
    #sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer=sgd, loss="mse", metrics=['mae', 'acc'])
    adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='mse', optimizer=adam)
    # checkpoint
    filePath = "/home/scope/Carla/CARLA_0.9.6/PythonAPI/new/model/weights.best.hdf5"
    callbacks_list = [
    ModelCheckpoint(filePath, monitor='val_mean_absolute_error', verbose=1, save_best_only=False),
    EarlyStopping(monitor='val_mean_absolute_error', patience=10, verbose=0),
    TensorBoard(log_dir='./tensorboard/', histogram_freq=0, write_graph=False, write_images=False)
    ]
#     checkpoint = ModelCheckpoint(filePath, monitor='loss', verbose=1, save_best_only=True, mode='min')
#     callbacks_list = [checkpoint, history]
    model.fit(X_train, Y_train, epochs=1000, batch_size=128,validation_data=(X_test, Y_test),callbacks=callbacks_list, verbose=2)
    

def saveModel(model):
	model_json = model.to_json()
	with open("/home/scope/Carla/CARLA_0.9.6/PythonAPI/new/model/model.json", "w") as json_file:
		json_file.write(model_json)

	model.save_weights("/home/scope/Carla/CARLA_0.9.6/PythonAPI/new/model/model.h5")
	print("Saved model to disk")

In [6]:
model = createModel()
print("create a new model")

TypeError: ('Keyword argument not understood:', 'border_mode')

In [ ]:
trainModel(model, image_input, steer_output)
print("completed training the model")

In [ ]:
saveModel(model)
